In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")


# Exploratory Data Analysis

In [ ]:
#dropping unwanted columns
train_eda = train.drop(["Name","Cabin","Ticket"],axis=1)

In [ ]:
train_eda.head()

In [ ]:
train_eda.shape

In [ ]:
train_eda.isnull().sum()

In [ ]:
train_eda.describe()

In [ ]:
train_eda.columns

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data = train_eda.corr(), annot = True, cmap = 'Blues')
plt.show()

#### Fare and Survived are the most corretalted others include Parch and Survived , We can use fare and parch features for predicting the survived

In [ ]:
train_eda.info()

In [ ]:
sns.boxplot(x=train_eda["Fare"])

In [ ]:
sns.boxplot(x=train_eda["Pclass"])
print(train_eda["Pclass"].mean())
print(train_eda["Pclass"].median())

In [ ]:
train_eda.isnull().sum()

# Cleaning Data

In [ ]:
#dropping empty rows
train_fin = train_eda.dropna()

In [ ]:
#checking if the rows are dropped
train_fin.isnull().sum()

In [ ]:
#checking shape are dropping
train_fin.shape

In [ ]:
#changing gender to numeric
train_fin.loc[train_fin.Sex=='female','Sex']=1
train_fin.loc[train_fin.Sex=='male','Sex']=0
train_fin["Sex"] = train_fin["Sex"].astype(str).astype(float)

#changing strings to numeric
train_fin.loc[train_fin.Embarked =='S','Embarked']= 3
train_fin.loc[train_fin.Embarked =='C','Embarked']=2
train_fin.loc[train_fin.Embarked =='Q','Embarked']=1
train_fin["Embarked"] = train_fin["Embarked"].astype(str).astype(float)
#dropping passengerid
train_fin = train_fin.drop(['PassengerId'],axis=1)

In [ ]:
#confirming the data
train_fin.head()

In [ ]:
#fixing test data
test['Age'] = test['Age'].fillna(test['Age'].mean())
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())
test.loc[test.Sex=='female','Sex']=1
test.loc[test.Sex=='male','Sex']=0
test["Sex"] = test["Sex"].astype(str).astype(float)
test.loc[test.Embarked =='S','Embarked']= 3
test.loc[test.Embarked =='C','Embarked']=2
test.loc[test.Embarked =='Q','Embarked']=1
test["Embarked"] = test["Embarked"].astype(str).astype(float)
test.isnull().sum()

In [ ]:
#dropping unwanted columns 
test = test.drop(["Name","Cabin","Ticket"],axis=1)
test.head()

In [ ]:
x_test = test.drop("PassengerId",axis=1)

In [ ]:
#setting variables for modelling
x = train_fin.drop('Survived', axis=1)
y = train_fin['Survived']

# Modelling


In [ ]:
model_dt = GradientBoostingClassifier(min_samples_leaf=5,max_leaf_nodes=5)
model_dt.fit(x,y)
pred_y = model_dt.predict(x)
acc = accuracy_score(y,pred_y)
test_pred = model_dt.predict(x_test)
print(acc)
xgboost = XGBClassifier(learning_rate=0.005,
                       n_estimators=6000,
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:squarederror',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)

xgboost.fit(x,y)
pred_xg = xgboost.predict(x)
ac = accuracy_score(y,pred_xg)
test_pred_xg = xgboost.predict(x_test)
print(ac)

In [ ]:
#x_reduced = []
#for i in range(1,419):
    #x_reduced.append(x[i])


In [ ]:
predictions = [round((test_predic + xg_pred) / 2) for test_predic,xg_pred in zip(test_pred,test_pred_xg)]
xgboost.fit(x_test,predictions)
predictio = xgboost.predict(x_test)
predictions = [round((test_predic + xg_pred) / 2) for test_predic,xg_pred in zip(test_pred,predictio)]
model_dt.fit(x_test,predictions)
pred1 = model_dt.predict(x_test)
predictions = [round((test_predic + xg_pred) / 2) for test_predic,xg_pred in zip(pred1,predictio)]
model_dt.fit(x_test,predictions)
xgboost.fit(x_test,predictions)
pred2 = model_dt.predict(x_test)
pred3 = xgboost.predict(x_test)
predictions = [round((test_predic + xg_pred) / 2) for test_predic,xg_pred in zip(pred2,pred3)]
model_dt.fit(x_test,predictions)
xgboost.fit(x_test,predictions)
pred4 = model_dt.predict(x_test)
pred5 = xgboost.predict(x_test)
predictions = [round((test_predic + xg_pred) / 2) for test_predic,xg_pred in zip(pred4,pred5)]
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)